Mongolian text classification series #01

In this notebook I'm gonna try to classify cyrillic mongolian texts using modern Tensorflow 2.0

Eduge dataset provided by Bolorsoft LLC

Author : Sharavsambuu Gunchinish (sharavsambuu@gmail.com)

Github: https://github.com/sharavsambuu/mongolian-text-classification 



In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

2.0.0-alpha0


[More info about creation of eduge dataset pickles](https://github.com/sharavsambuu/mongolian-text-classification/blob/master/preprocess_dataset/preprocess_eduge.ipynb) 

In [2]:
import os
from os.path import exists, join, basename, splitext
import sys

def download_from_google_drive(file_id, file_name):
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  
# download eduge pickles
file_path = 'eduge_pickles'
if not exists(file_path):
  download_from_google_drive('1vjJ9YgIe8o0ErhbN0lH1XqPv3KFP8acv', '%s.rar' % file_path)
  rar_file = file_path+".rar"
  !unrar x $rar_file

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   3079      0 --:--:-- --:--:-- --:--:--  3079
100  106M    0  106M    0     0   130M      0 --:--:-- --:--:-- --:--:--  221M

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from eduge_pickles.rar


Would you like to replace the existing file word_index.pickle
9178153 bytes, modified on 2019-04-13 01:44
with a new one
9178153 bytes, modified on 2019-04-13 01:44

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit y

Extracting  word_index.pickle                                              2%  OK 

Would you like to replace the existing file eduge.pickle
359611555 bytes, modified on 2019-04-13 01:44
with a new one
359611555 bytes, modified on 2019-04-13 01:44

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit A

Extracting  eduge.pickle                                 

In [0]:
import pickle

with open('word_index.pickle', 'rb') as handle:
  word_index = pickle.load(handle)
    
with open('reversed_word_index.pickle', 'rb') as handle:
  reversed_word_index = pickle.load(handle)
  
with open('eduge_stopwords_removed.pickle', 'rb') as handle:
  eduge_ds = pickle.load(handle)

In [0]:
MAX_LEN = 512

import itertools

for item in eduge_ds:
  item[0] = list(itertools.chain(*item[0]))[:MAX_LEN]

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(eduge_ds, test_size=0.1, random_state=999)

In [0]:
train_data_words  = [i[0] for i in train]
train_label_words = [i[1] for i in train]
test_data_words   = [i[0] for i in test ]
test_label_words  = [i[1] for i in test ]

In [0]:
def encode_news(text):
    return [word_index.get(i, 2) for i in text]
  
train_data = [encode_news(sent) for sent in train_data_words]
test_data  = [encode_news(sent) for sent in test_data_words ]

In [0]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=MAX_LEN)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=MAX_LEN)

In [9]:
labels = list(set(test_label_words))
labels

['урлаг соёл',
 'эдийн засаг',
 'хууль',
 'улс төр',
 'технологи',
 'боловсрол',
 'байгал орчин',
 'спорт',
 'эрүүл мэнд']

In [0]:
from sklearn.preprocessing import LabelBinarizer
encoder     = LabelBinarizer()
train_label = transfomed_label = encoder.fit_transform(train_label_words)
test_label  = transfomed_label = encoder.fit_transform(test_label_words )

In [11]:
vocab_size = len(word_index)

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(len(labels), activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          5932704   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 153       
Total params: 5,933,129
Trainable params: 5,933,129
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
print(len(train_data), len(train_label))
print(len(test_data ), len(test_label) )

partial_index = 3000

x_val = train_data[:partial_index]
partial_x_train = train_data[partial_index:]

y_val = train_label[:partial_index]
partial_y_train = train_label[partial_index:]

68094 68094
7567 7567


In [14]:
epochs = 30
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 65094 samples, validate on 3000 samples
Epoch 1/30
65094/65094 [==============================] - 15s 235us/sample - loss: 2.1538 - accuracy: 0.2843 - val_loss: 2.0913 - val_accuracy: 0.2657
Epoch 2/30
65094/65094 [==============================] - 15s 229us/sample - loss: 1.9881 - accuracy: 0.2770 - val_loss: 1.8629 - val_accuracy: 0.3077
Epoch 3/30
65094/65094 [==============================] - 15s 225us/sample - loss: 1.6950 - accuracy: 0.3745 - val_loss: 1.5445 - val_accuracy: 0.4200
Epoch 4/30
65094/65094 [==============================] - 15s 230us/sample - loss: 1.3951 - accuracy: 0.5011 - val_loss: 1.2911 - val_accuracy: 0.5603
Epoch 5/30
65094/65094 [==============================] - 15s 230us/sample - loss: 1.1536 - accuracy: 0.6413 - val_loss: 1.0813 - val_accuracy: 0.6913
Epoch 6/30
65094/65094 [==============================] - 15s 232us/sample - loss: 0.9536 - accuracy: 0.7518 - val_loss: 0.9165 - val_accuracy: 0.7713
Epoch 7/30
65094/65094 [=====================

In [15]:
results = model.evaluate(test_data, test_label)
print(results)

7567/7567 [==============================] - 0s 52us/sample - loss: 0.4103 - accuracy: 0.8936
[0.4103335821194946, 0.89361703]


In [16]:
data_index   = 12
data_words   = " ".join(test_data_words[data_index])
data_indexes = test_data[data_index]
print(data_words)
#print(data_indexes)
import numpy as np
predicted = model.predict([data_indexes])
print(encoder.classes_[np.argmax(predicted)])

спортын төв ордонд өнөөдөр азийн оюутны аварга шалгаруулах эмэгтэй волейболчдын хоёр дахь удаагийн тэмцээний талаар мэдээлэл хийлээ анхны тэмцээн онд тайландын бангконг хотноо болж хоёрдугаар тэмцээнийг азийн оюутны спортын холбооноос аосх олгосон эрхийн дагуу оны дөрөвдүгээр сарын ны өдрүүдэд монгол улсын нийслэл улаанбаатар хотноо зохион байгуулах тэмцээний эрхийг монгол улс оны тавдугаар сарын хуралдсан аосхны гүйцэтгэх хорооны хурлаар хоёр оронтой өрсөлдөн авчээ уг тэмцээнийг монгол улсад авах талаар мосхолбоо оноос санаачлага гарган хөөцөлдөж эхэлсэн тэмцээний эрхийг авахад муын засгийн газрын санхүүгийн дэмжлэг мэргэжлийн холбоодын ажлын туршлага манай улсын олон улсын нэр хүнд ихээхэн тус хүргэжээ зохион байгуулах хороог с ламбаа удирдаж тэмцээний зохион байгуулах хороог збх эрүүл мэндийн сайдын оны тоот тушаалаар батлаж даргаар уихын гишүүн монголын волейболын холбооны мвх хүндэт ерөнхийлөгч сламбаа ажиллаж збхны орлогч даргаар згхагентлагбтсгын дарга чнаранбаатар збхны нарийн 